# トレーニングモデル

機械学習の中心的な目標は、アプリケーションで使用できる予測モデルをトレーニングすることです。 Azure Machine Learningでは、スクリプトを使用して、Scikit-Learn、Tensorflow、PyTorch、SparkMLなどの一般的な機械学習フレームワークを活用したモデルをトレーニングできます。これらのトレーニングスクリプトを実験として実行して、メトリックと出力（特に、トレーニングされたモデル）を追跡できます。

## ワークスペースに接続する

最初に行う必要があるのは、Azure ML SDKを使用してワークスペースに接続することです。

> **Note**: 前の演習を完了してから、Azureサブスクリプションとの認証済みセッションの有効期限が切れた場合、再認証するように求められます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## トレーニングスクリプトを作成する

Pythonスクリプトを使用して、diabatesデータに基づいて機械学習モデルをトレーニングするため、スクリプトとデータファイル用のフォルダーを作成することから始めましょう。

In [ ]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'diabetes-training'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

これで、トレーニングスクリプトを作成してフォルダーに保存する準備ができました。

In [ ]:
%%writefile $training_folder/diabetes_training.py
# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Set regularization hyperparameter
reg = 0.01

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the trained model in the outputs folder
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## 推定器を使用して、実験としてスクリプトを実行する

以前は、**RunConfiguration**および**ScriptRunConfig**を使用して実験スクリプトを実行しました。同じアプローチを使用してトレーニングスクリプトを実行できますが、**Estimator**を使用すると、これらの両方の構成が1つのオブジェクトに抽象化されるため、一般的に簡単です。

この場合、一般的な**Estimator**オブジェクトを使用してトレーニング実験を実行します。この推定器のデフォルト環境には**scikit-learn**パッケージが含まれていないため、明示的に構成に追加する必要があることに注意してください。 conda環境は、推定器が初めて使用されるときにオンデマンドで構築され、同じ構成を使用する将来の実行のためにキャッシュされます。そのため、最初の実行には少し時間がかかります。

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment

# Create an estimator
estimator = Estimator(source_directory=training_folder,
                      entry_script='diabetes_training.py',
                      compute_target='local',
                      conda_packages=['scikit-learn']
                      )

# Create an experiment
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment based on the estimator
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=True)

実験の実行と同様に、**RunDetails**ウィジェットを使用して、実行に関する情報を表示し、Azure Machine Learning Studioで実行へのリンクを取得できます。

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

**Run**オブジェクトからメトリックと出力を取得することもできます。

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## トレーニング済みモデルを登録する

実験の出力には、トレーニング済みのモデルファイル（**diabetes_model.pkl**）が含まれていることに注意してください。このモデルをAzure Machine Learningワークスペースに登録して、モデルのバージョンを追跡し、後で取得できるようにすることができます。

In [ ]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Estimator'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## パラメーター化されたトレーニングスクリプトを作成する

スクリプトにパラメーターを追加することにより、トレーニング実験の柔軟性を向上させることができ、異なる設定で同じトレーニング実験を繰り返すことができます。この場合、モデルのトレーニング時にロジスティック回帰アルゴリズムで使用される正則化率のパラメーターを追加します。

繰り返しますが、パラメーター化されたスクリプトとトレーニングデータ用のフォルダーを作成することから始めましょう。

In [ ]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'diabetes-training-params'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

次に、正則化率ハイパーパラメーターのパラメーターを含むスクリプトを作成しましょう。

In [ ]:
%%writefile $training_folder/diabetes_training.py
# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Get the experiment run context
run = Run.get_context()

# Set regularization hyperparameter
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, dest='reg', default=0.01)
args = parser.parse_args()
reg = args.reg

# load the diabetes dataset
print("Loading Data...")
# load the diabetes dataset
diabetes = pd.read_csv('diabetes.csv')

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## フレームワーク固有の見積もりツールを使用する

汎用**Estimator**クラスを使用してトレーニングスクリプトを実行しましたが、一般的な機械学習フレームワークの環境定義を含むフレームワーク固有の推定器を利用することもできます。この場合、Scikit-Learnを使用しているため、**SKLearn**推定器を使用できます。これは、設定で**scikit-learn**パッケージを指定する必要がないことを意味します。

In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.widgets import RunDetails

# Create an estimator
estimator = SKLearn(source_directory=training_folder,
                    entry_script='diabetes_training.py',
                    script_params = {'--reg_rate': 0.1},
                    compute_target='local'
                    )

# Create an experiment
experiment_name = 'diabetes-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment
run = experiment.submit(config=estimator)

# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

繰り返しますが、完了した実行からメトリックと出力を取得できます。

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## モデルの新しいバージョンを登録する

新しいモデルをトレーニングしたので、ワークスペースに新しいバージョンとして登録できます。

In [ ]:
from azureml.core import Model

# Register the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Parameterized SKLearn Estimator'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

探索が終了したら、このノートブックを閉じてコンピューティングインスタンスをシャットダウンできます。